导入运行库

In [39]:
import pandas as pd
import numpy as np

时间+评分

In [40]:
def compute_store_ema(df, time_col="timestamp", score_cols=["taste", "speed", "service"], k=0.2):
    """
    输入：包含 timestamp + 多个评分维度的DataFrame
    输出：每个维度的最终 EMA 值（时间衰减）
    k 为时间衰减系数（越大越强调近期评分）
    """

    df = df.copy()
    df[time_col] = pd.to_datetime(df[time_col])
    df = df.sort_values(time_col).reset_index(drop=True)

    # 初始化 EMA
    ema = {col: df.loc[0, col] for col in score_cols}

    # 遍历每一条评分记录
    for i in range(1, len(df)):
        time_diff_days = (df.loc[i, time_col] - df.loc[i-1, time_col]).total_seconds() / 86400
        alpha = 1 - np.exp(-k * time_diff_days)  # 动态衰减因子

        for col in score_cols:
            ema[col] = alpha * df.loc[i, col] + (1 - alpha) * ema[col]

    return ema

读取生成的数据

In [41]:
df = pd.read_csv('./Result_Data/天天煲仔饭_result.csv')

输出EMA平均值

In [42]:
final_ema = compute_store_ema(df, k=0.3)
print("最终三个维度的EMA平均：")
print(final_ema)

最终三个维度的EMA平均：
{'taste': -0.8996872420309765, 'speed': -0.29992055761411485, 'service': -0.6998313155190056}
